In [11]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from tensorflow.keras.layers import Dense,Input,GlobalMaxPooling1D,LSTM,Embedding,Conv1D,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
train_data=pd.read_csv('/kaggle/input/anyas-gojo-revival-space-shop/train.tsv', sep='\t')
train_data.head()

,date,time,id,reviewer name,reviewer address,contact number,variation,verified_reviews,sentiment,feedback
0,30-Jul-18,02:16:00,5138527,Theresa Diaz,"2373 Long Mews\nSouth Lorraine, FL 83715",7445448464,Black Plus,Love my new Echo Plus,5,1
1,30-Jul-18,20:13:24,7881409,Michael Wagner,"34120 James Well Apt. 930\nSouth Kenneth, AK 5...",(202)233-6662,Configuration: Fire TV Stick,Works really well.,5,1
2,29-Jul-18,03:20:48,5658396,Erin Rodriguez,"7996 Booth Mission\nTaylorport, AL 13457",(219)882-0705,Black Show,Echo Show is said to work with certain apps bu...,2,0
3,30-Jul-18,22:48:21,4317957,Christine Perez,USS Schaefer\nFPO AA 86432,291-900-6087,Heather Gray Fabric,Very cool product. Speaker sounds good with my...,5,1
4,23-Jul-18,12:36:42,7570569,Jamie Davila,"000 Kevin Cliffs Suite 674\nBrennanview, NM 77690",9527296610,Black Plus,So far I love it. It was easy to set up - if ...,5,1


In [3]:
train_data.isnull().sum()

date                0
time                0
id                  0
reviewer name       0
reviewer address    0
contact number      0
variation           0
verified_reviews    1
sentiment           0
feedback            0
dtype: int64

In [4]:
train_data.fillna({"verified_reviews":"no review"}, inplace=True)

In [5]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [6]:
from scipy.special import softmax
pov_review=[]
neg_review=[]
neu_review=[]
for i in train_data['verified_reviews']:
    try:
        encoded_text = tokenizer(i, return_tensors='pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
#         print(scores)
        neg_review.append(scores[0])
        neu_review.append(scores[1])
        pov_review.append(scores[2])
    except RuntimeError:
        print(1)
        pov_review.append(1.0)
        neg_review.append(0.0)
        neu_review.append(0.0)

1
1


In [88]:
new_dataframe=pd.DataFrame({"positive":pov_review,"negative":neg_review,"neutral":neu_review,"feedback":train_data['feedback']})
new_dataframe.head()

,positive,negative,neutral,feedback
0,0.989508,0.001466,0.009026,1
1,0.936137,0.004737,0.059127,1
2,0.107922,0.570545,0.321533,0
3,0.988388,0.001345,0.010267,1
4,0.991431,0.002093,0.006476,1


In [89]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(new_dataframe, train_data['sentiment'], random_state = 0) 
dtree_model = RandomForestClassifier(n_estimators=1000 ,random_state=42).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

In [90]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
y_train = y_train.to_numpy()
encoder = OneHotEncoder(sparse=False, categories='auto')
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
def build_model():
    model = Sequential([
        Dense(128, activation='relu',input_dim=4),
        Dense(64,activation='relu'),
        Dense(32,activation='relu'),
        Dense(5, activation='softmax')  # Output layer with 'softmax' for multi-class classification
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model
estimator = KerasClassifier(build_fn = build_model, epochs = 10, batch_size = 20, verbose = 1)
sscaler = StandardScaler()
X_train = sscaler.fit_transform(X_train)
estimator.fit(X_train, y_train_encoded)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/tmp/ipykernel_32/4161819188.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = build_model, epochs = 10, batch_size = 20, verbose = 1)


95/95 [==============================] - 1s 2ms/step - loss: 0.3348 - accuracy: 0.7693
Epoch 2/10
95/95 [==============================] - 0s 2ms/step - loss: 0.2045 - accuracy: 0.7873
Epoch 3/10
95/95 [==============================] - 0s 2ms/step - loss: 0.2000 - accuracy: 0.7937
Epoch 4/10
95/95 [==============================] - 0s 2ms/step - loss: 0.1992 - accuracy: 0.7942
Epoch 5/10
95/95 [==============================] - 0s 2ms/step - loss: 0.1979 - accuracy: 0.7937
Epoch 6/10
95/95 [==============================] - 0s 2ms/step - loss: 0.1980 - accuracy: 0.7926
Epoch 7/10
95/95 [==============================] - 0s 2ms/step - loss: 0.1972 - accuracy: 0.7926
Epoch 8/10
95/95 [==============================] - 0s 2ms/step - loss: 0.1974 - accuracy: 0.7937
Epoch 9/10
95/95 [==============================] - 0s 2ms/step - loss: 0.1979 - accuracy: 0.7926
Epoch 10/10
95/95 [==============================] - 0s 2ms/step - loss: 0.1964 - accuracy: 0.7937


In [85]:
from sklearn.metrics import confusion_matrix 
accuracy = model.predict(X_test)
print(accuracy)

20/20 [==============================] - 0s 2ms/step
[[0.23793797 0.21166357 0.19784862 0.16500676 0.18754305]
 [0.23063584 0.19416122 0.21925302 0.1579343  0.19801553]
 [0.24062191 0.21238232 0.1970858  0.16404982 0.18586002]
 ...
 [0.22962308 0.19491054 0.21614055 0.15795542 0.20137036]
 [0.22947037 0.19121401 0.22963941 0.15299045 0.19668585]
 [0.228596   0.19854744 0.21129933 0.16030726 0.20124991]]


In [14]:
test_data=pd.read_csv('/kaggle/input/anyas-gojo-revival-space-shop/test.tsv', sep='\t')
pov_test_review=[]
neg_test_review=[]
neu_test_review=[]
for i in test_data['verified_reviews']:
    try:
        encoded_text = tokenizer(i, return_tensors='pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
#         print(scores)
        neg_test_review.append(scores[0])
        neu_test_review.append(scores[1])
        pov_test_review.append(scores[2])
    except RuntimeError:
        print(1)
        pov_test_review.append(1.0)
        neg_test_review.append(0.0)
        neu_test_review.append(0.0)

In [27]:
test_dataframe=pd.DataFrame({"positive":pov_test_review,"negative":neg_test_review,"neutral":neu_test_review,"feedback":test_data['feedback']})
test_dataframe.head()

,positive,negative,neutral,feedback
0,0.984899,0.002404,0.012697,1
1,0.986175,0.003311,0.010514,1
2,0.005175,0.963042,0.031783,0
3,0.858581,0.032428,0.108991,1
4,0.963731,0.003297,0.032972,1


In [28]:
arr=dtree_model.predict(test_dataframe)
print(arr)

[5 5 1 5 5 5 5 5 1 3 5 5 5 4 5 5 1 5 3 4 5 5 5 5 5 3 1 5 5 1 5 5 5 5 5 5 5
 5 5 5 5 1 5 5 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 1 5 5 5 5 1 5 3 5 5 5 5 2 5 5 5 4 5 5 5 5 5 4 1 5 1 5 4 5 5 5 5
 5 2 5 5 5 5 5 5 5 5 4 5 5 3 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5 5 5
 5 3 5 5 4 5 5 5 5 4 5 2 5 5 3 4 5 5 5 5 5 5 5 5 5 5 4 5 5 2 5 5 5 5 5 2 5
 2 5 5 4 5 5 5 5 5 5 5 4 5 5 5 5 5 5 4 5 5 3 1 5 5 4 5 5 5 4 5 5 4 5 5 5 5
 5 5 5 5 5 5 1 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 3 5 5 3 3 1 1 5 4 5 5 5 2 5 5
 5 4 3 5 5 5 4 5 5 5 5 3 5 5 1 5 5 5 5 3 5 4 3 3 5 5 5 5 5 4 4 4 5 5 5 5 5
 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 1 3 4 5 5 5 5 5 1 5 5 5 5 5 1 5 5 5 4 4 4 5 5 4 5 5 5 4
 5 5 5 5 2 5 5 5 5 5 1 5 5 5 5 5 5 1 1 3 5 5 5 5 2 3 5 5 1 5 5 5 5 5 5 5 5
 5 5 5 5 4 5 4 5 5 5 5 2 5 3 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 2 3 5
 3 5 5 4 5 2 5 5 5 1 4 5 5 5 5 5 5 5 5 4 5 5 1 5 5 5 5 5 5 5 5 4 2 5 5 5 5
 5 5 5 3 5 5 5 5 4 1 4 5 

In [29]:
submission=pd.DataFrame({"id":test_data['id'],"sentiment":arr})
submission.to_csv("submission6.csv",index=False)